# Кредитоспособность

Определим влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

Рельтаты будут использованы для построения модели кредитного скоринга.

## 1. Обзор данных

Составим первое представление о данных.

Импортируем библиотеки:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

Считываем исходный файл с данными `creditworthiness.csv`:

In [2]:
df = pd.read_csv('./datasets/creditworthiness.csv')

Выводим первые 7 строк:

In [3]:
df.head(7)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем


Выведим общую информацию о структуре данных:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Размерность таблицы 21525×12.

Согласно документации к данным:

   - `children` - количество детей в семье
   - `days_elployed` - общий трудовой стаж в днях
   - `dob_years` - возраст клиента в годах
   - `education` - уровень образования клиента
   - `family_status` - семейное положение
   - `family_status_id` - код семейного пложения
   - `gender` - пол клиента
   - `income_type` - тип занятости; должность
   - `debt` - наличие долговых обязательств
   - `total_income` - ежемесячный доход
   - `purpose` - цель получения кредита


**Обобщение**

Каждая строка - это данные о заёмщике.

Количество данных почти во всех столбцах одинаково и максимально, исключая `days_employed` и `total_income`, в которых оно одинаково и меньше максимального. Можно предположить о пропусках в столбцах строк, которые описывают клиента, не имеющего опыта работы и, как следствие, ежемесячного заработка.

Для дальнейших исследований необходимо устранить проблемы в данных.

## 2. Предобработка данных

### 2.1. Поле `children`

Проверим на наличие аномалий столбец `children`. Выведим все возможные значения этого столбца и посчитаем общее количество строк для каждого из них:

In [5]:
# fig, ax = plt.subplots()
# ax.bar(df['children'].unique(), df['children'].value_counts())
# ax.grid()
# ax.set_xlabel('Количество детей')
# ax.set_ylabel('Количество заёмщиков')
# plt.xticks(np.arange(df['children'].min(), df['children'].max() + 1))
# plt.show()

In [6]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Обнаружено 2 аномальных значения:

- **-1**. Очевидно, что такого количества детей быть не может, и учитывая малое количество таких значений, всего 47 штук, можно выдвинуть следующие предположения о природе их появления: опечатки при внесение данных, ошибка при составление выгрузки. При внимательном рассмотрение первых строк из выгрузки в пункте 1, можно заметить, что столбец `days_employed` так же содержит отрицательные значения. Предположим, что отрицательные значения вызваны ошибкой при составление выгрузки, и искомыми значениями являются эти же значения по абсолютной величине. Таким образом, изменим эти значения на значения второй по многочисленности группы, заёмщики с одним ребёнком. Количество этой группы возрастёт на *0.1%*. Такое приведение к многочисленной группе повлечёт незначительные изменения результатов исследований

In [7]:
df['children'].replace(-1, 1, inplace=True)

- **20**. Заёмщики с таким количество детей возможны, но стоит учесть совокупность фактов, ставящих под сомнение корректность этих полей: количество заёмщиков с 4 детьми почти в 2 раза меньше чем с 20, отсутствуют заёмщики с количеством детей от 6 до 19. Предположения о природе появления данных значений аналогичны прошлому пункту, однако возможно, что *среди них присутствуют корректные, которые были много кратно продублированы*. После приведения всех "текстовых" полей к нижнему регистру, в блоке `2.. Устранение дубликатов` необходимо провести проверку гипотезы.

Выполним проверку изменений в данных:

In [8]:
df['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

**Выводы:** обнаружены малочисленные аномальыне значения `-1` и `20`. Общими причинами появления данных аномалий могут быть опечатки при внесение данных и ошибки при составление выгрузки: для '-1' - искажение значений, для '20' - дублирование одной или нескольких корректных строк. Исходя из перечисленных причин принят следующий ряд решений:

- значения `-1` замененили на многочисленное `1`, так как значение `-1` в общем случае не является корректным для количества детей, и приведение к многочисленной группе повлечёт незначительные изменения результатов исследований (прирост количества заёмщиков в группе с 1 ребйнком после приведения составил *0.9%*),

- значение `20` в общем случае является возможным для данного поля, поэтому необходимо проверить гипотезу о дублирование одной или нескольких строк с таким значением. Гипотеза будет проверена в блоке `2.. Устранение дубликатов`, так как необходимо привести все "текстовые" столбци к нижнему регистру, чтобы обнаружить все дубликаты. При подтверждение гипотезы после выше упомянутого блока все дубликаты будут удалены, иначе значения будут преведены к `0` по причинам изложенным в прошлом пункте.


При принятие решения о изменение аномальных значений на самыое многочисленное были приняты во внимания все задачи данного исследования - столбец `children` необходим для решения *только одной* задачи.

### 2.2. Поле `days_employed`

#### 2.2.1. Обработка аномальных значений

При подробном рассмотрение частичного представленния данных в пункте 1, можно заметить, что поле `days_employed` содержит отрицательные значения. Одними из причинами появления данных значений могут быть: ошибка ввода данных и ошибка создания выгрузки. Ознакомимся со строками, содержащими даннные значения:

In [9]:
df[df['days_employed'] < 0].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья


In [10]:
df.loc[df['days_employed'] < 0, 'days_employed'].describe()

count    15906.000000
mean     -2353.015932
std       2304.243851
min     -18388.949901
25%      -3157.480084
50%      -1630.019381
75%       -756.371964
max        -24.141633
Name: days_employed, dtype: float64

Видно, что строки с отрицательными значениями в столбце `days_employed` составляют **74%** от общего количества строк и представляют большое разнообразие значений. Учитывая аналогичные проблемы со значениями в столбце `children`, но в меньшем количестве, и сходство значений с корректными, но с противоположным знаком. Предположим, что эти аномалии вызваны ошибкой создания выгрузки; представления искомых значений в противоположном по знаку виде. Приведём отрицательные значения к абсолютной величине.

In [11]:
df['days_employed'] = df['days_employed'].apply(abs)

Выполним проверку, выведя описательную статистику поля `days_employed`:

In [12]:
df['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

Минимальное значение больше 0 - приведение прошло успешно.

**Выводы:** обнаружено большое количество *74%* строк, содержащих отрицательные значения в столбце с заведомо неотрицательными данными. В этого факта и того, что все отрицательные значения очень разнообразны от -18389 до -24 и имеют неравномерное распределение, было выдвинуто предположение о том, что эти аномалии вызваны ошибкой составления выборки, которая проявилась. Ошибка привела к замене большей части значений столбца `days_employed` на им отрицательные. Таким образом, отрицательные значения были приведены к абсолютным величинам.

#### 2.2.2. Обработка пропусков

Из представления данных так же выдны пропуски данного поля, посчитаем их долю относительно всех значений:

In [1]:
fig, ax = plt.subplots()

# Вычисляем долю пропусков.
na_fraction = df['days_employed'].isna().sum() / df.shape[0]

# Вычисляем значения для задания диаграммы.
values = [na_fraction, 1 - na_fraction]
labels = ['NA', 'Corrected']
colors = [plt.get_cmap('Purples')(0.55), plt.get_cmap('YlOrBr')(0.3)]

ax.set_title('Доля пропусков поля "days_employed"')
ax.pie(
    values,
    labels=labels,
    autopct='%1.f%%',
    colors=colors
)


plt.show()

NameError: name 'plt' is not defined

Из графического представления видно, что пропуски составляют большую долю всех значений (*10%*). В данном случае причинами могут быть, например, ошибки ввода или создания выгрузки. Оба варианта подразумевают, что данные невозможно восстановить в первоначальном виде, однако удаление такого количества строк пагубно скажется на точности результатов исследований. Заполним пропуски медианными значениями поля `days_employed` для соответствующей группы вида деятельности заёмщика `income_type`. Такое решение позволит использовать данные поля `days_employed` для дальнейших исследований не нарушаю статистических значений, так как медиана сохраняет независимость от аномально больших или малых значений.

Создадим функцию заполняющую пропуски в 'A' столбце медианными значениями групп, созданных по столбцу 'B':

In [14]:
def fill_na_median_by_group(values: str,  group_by: str):
    medians_by_groups = df.groupby(group_by)[values].agg('median').reset_index()
    
    for row_index in range(medians_by_groups.shape[0]):
        df.loc[
            (df[group_by] == medians_by_groups.loc[row_index, group_by]) & \
            df[values].isna(),
            values
        ] = medians_by_groups.loc[row_index, values]
            

Заполним пропуски медианными значениями поля `days_employed` для соответствующей группы вида деятельности заёмщика `income_type` и выполним проверку количества пропусков в поле `days_employed`:

In [15]:
fill_na_median_by_group('days_employed', 'income_type')

# Выводим количество пропусков столбца 'days_employed'.
df['days_employed'].isna().sum()

0

**Выводы:** данные содержат большой процент строк с пропусками в поле `days_employed` *10%*. Предположительная природа пропусков - это ошибка создания выгрузки. Так как удаление такого процента данных преведёт к искажению результатов исследований, было принято решение о заполнение пропусков медианными значениями, полученными по группе соответствующего типа дохода заёмщика `income_type`. Такаи образом, заполненные значения не будут иметь негативного влияния на исследования связанные с данным столбцом, так как медиана является значением невосприимчивым к аномальным значениям, а сохранившиеся значения остальных полей поспособствуют увеличению точности исследования в целом.

#### 2.2.3. Распределение медианы по типу доходов

Для каждого типа занятости `income_type` выведим медианное значение общего стажа `days_employed`:

In [35]:
df.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

*Выводы:* у двух типов(пенсионеров и безработных) общий рабочий стаж значительно больше любого типа. Так как данный столбец `days_employed` не понодобится для дальнейших исследований, оставим эти данные в таком виде.

### <font color='blackgreen'>Комментарий</font>

<font color='blackgreen'>Важно ссоблюдать последовательность обработки столбцов. В противном случае, исследование начинает "скакать" и тебе самому будет не понятно, какие столбцы были отработаны, а какие нет.</font>

<font color='blackgreen'>Необходимо отработать каждое отклонение по отдельности и сформировать предположения, по каким причине могли возникнуть подобные отклонения. После формирования предположение, можно будет принимать обоснованное решение, как обрабатывать отклонения.</font>

<font color='red'>Обрати внимание, что по этим столбцам, у тебя отсутсвтуют выводы. По-хорошему, у тебя должны быть выводы по итогам анализа каждого столбца.</font>

### 2.2. Устарение пропусков

Пропущенные значения в столбцах `days_employed` и `total_income` заполним медианами доходов для соответствующих типа занятости `income_type`:

<font color='red'>В этом разделе, так не понятно как ты пришёл к этим выводам, по заполнению пропусков медианным значением. Так же требуется обоснование, почему используется именно медианное значение.</font>

<font color='red'>Важно контролировать проводимые преобразования, что было <b>до</b> изменений и что получилось после внесения изменений. В данном примере ты контролируем только итог преобразования, а что было ДО у тебя отсутствует. Так же отсутстсвуют выводы, которые касаются результата внесения изменений.</font>

In [36]:
for income_type in df['income_type'].unique():
    # Восполняем данные в поля `days_employed`
    df.loc[(df['income_type'] == income_type) & df['days_employed'].isna(), 'days_employed'] = \
    df.loc[(df['income_type'] == income_type), 'days_employed'].median()

    # Восполняем данные в поля `total_income`
    df.loc[(df['income_type'] == income_type) & (df['total_income'].isna()), 'total_income'] = \
    df.loc[(df['income_type'] == income_type), 'total_income'].median()

Выполним проверку:

In [37]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### 2.3. Изменение типов данных

Для поставленных задач нет необходимости в высокой точности ежемесячного дохода `total_income`, поэтому приведём данное поле к целочисленному типу:

<font color='red'>Не понятное обоснование, скорее похоже на констатацию факта, а не обоснованное принятие решение.</font>

In [38]:
df['total_income'] = df['total_income'].astype(int)

### 2.4. Обработка дубликатов

Сперва приведём все данные столбцы `education` и `purpose` к нижнему регистру, чтобы привести все данные к однородности стиля. Это необходимо для поиска неявных дубликатов.

<font color='red'>Было бы не плохо проверить, может есть лишние пробелы/пропуски, их наличие так же негативным образом может повлиять на поиск неявных дубликатов.</font>

In [39]:
# Обраобтка столбца `education`
df['education'] = df['education'].str.lower()

# Обраобтка столбца `purpose`
df['purpose'] = df['purpose'].str.lower()

Определим количество явных дубликатов:

In [40]:
df.duplicated().sum()

71

Удалим дублирующиеся строки и обновим сместившиюся индексацию:

In [41]:
df = df.drop_duplicates().reset_index(drop=True)

### 2.5. Категоризация данных

Для дальнейших исследований введём категории на ежемесячный доход `total_income`:

- 0-30,000 - `'E'`;
- 30,001-50,000 - `'D'`;
- 50,001-200,000 - `'C'`;
- 200,001-1,000,000 - `'B'`;
- свыше 1,000,000 - `'A'`

Создадим функцию определяющую категорию ежемесячного дохода по входному числу:

In [42]:
def categorize_income(income: int) -> str:
    # Проверка входного значения.
    if type(income) != int:
        return None

    if income < 30001:
        return 'E'
    elif income < 50001:
        return 'D'
    elif income < 200001:
        return 'C'
    elif income < 1000001:
        return 'B'
    else:
        return 'A'

Воспользуемся функцией и категоризируем все строки, добавив новый столбец `total_income_category`:

In [43]:
df['total_income_category'] = df['total_income'].apply(categorize_income)

Выполним проверку:

In [44]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C


Теперь исследуем цели взятия кредита `purpose`:

In [45]:
purposes_unique = df['purpose'].unique()
purposes_unique.sort()

for purpose in purposes_unique:
    print(purpose)

автомобили
автомобиль
высшее образование
дополнительное образование
жилье
заняться высшим образованием
заняться образованием
на покупку автомобиля
на покупку подержанного автомобиля
на покупку своего автомобиля
на проведение свадьбы
недвижимость
образование
операции с жильем
операции с коммерческой недвижимостью
операции с недвижимостью
операции со своей недвижимостью
покупка жилой недвижимости
покупка жилья
покупка жилья для сдачи
покупка жилья для семьи
покупка коммерческой недвижимости
покупка недвижимости
покупка своего жилья
получение высшего образования
получение дополнительного образования
получение образования
приобретение автомобиля
профильное образование
ремонт жилью
свадьба
свой автомобиль
сделка с автомобилем
сделка с подержанным автомобилем
строительство жилой недвижимости
строительство недвижимости
строительство собственной недвижимости
сыграть свадьбу


Выделим основные группы целей:

- `получение образования`,
- `операциии с недвижимостью`,
- `операции с автомобилями`,
- `проведение свадьбы`

Создадим функцию определяющую абстрактную категорию цели `purpose`:

In [46]:
def categorize_purpose(purpose: str) -> str:
    if type(purpose) != str:
        return None

    if 'образов' in purpose:
        return 'получение образования'
    elif 'недвиж' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    elif 'авто' in purpose:
        return 'операции с автомобилями'
    elif 'свад' in purpose:
        return 'проведение свадьбы'
    else:
        return 'нет категории'

Воспользуемся функцией и категоризируем все строки добавив столбец `purpose_category`:

In [47]:
df['purpose_category'] = df['purpose'].apply(categorize_purpose)

Выполним проверку:

In [48]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилями
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


<font color='red'>В целом исследование хорошее, но стоит обратить внимание на тот фак, что тобой не проведён анализ и аномалий по всем столбцам DataFrame. Для всестороннего анализа всех возможный отклонений, стоит проводить анализ всех столбцов, как бы утомительно это не было. Как показывает практика, именно такой подход, поможет обеспечить выявление всех аномалий в данных.</font>

## 3. Исследования

### 3.1. Есть ли зависимость между количеством детей и возвратом кредита в срок?

Сгруппируем данные по количеству детей `children`, а также посчитаем общее количество заёмщиков, количество просроченных кредитов и отношение второго к первому:

In [49]:
df.pivot_table(index = 'children', values = 'debt', aggfunc = ['count', 'sum', 'mean']) \
    .sort_values(by = ('mean', 'debt'), ascending = False) \
    .style.format({('mean', 'debt'): '{:.2%}'})

,count,sum,mean
,debt,debt,debt
children,,,
20,76,8,10.53%
4,41,4,9.76%
2,2052,194,9.45%
1,4855,445,9.17%
3,330,27,8.18%
0,14091,1063,7.54%
5,9,0,0.00%


**Вывод:** 

1. Заёмщиков с 4-мя детьми, являются самыми `не надежними` по платёжной дисциплине (9,8%).
2. Самыми `надёжными` по возврату кредитов являются заёмщики не имеющие детей (7,5%).

Обращает на себя внимание тот факт, что у заёмщиков, имеющих 5 детей, просрочка отсутсвтует. Однако, с учётом доли просрочек заёмщиков с 4-мя детьми, будет ошибочным полагать, что у заёмщиков с 5-ю детьми доля просрочек будет сильно отличаться. Предполагаем, что такие показатели у заёмщиков с 5 детьми, обусловлены недостаточным количеством данных о выданных кредитах.

### 3.2. Есть ли зависимость между семейным положение и возвратом кредита в срок?

Сгруппируем данные по семейному положению `family_status`, а также посчитаем общее количество заёмщиков, количество просроченных кредитов и отношение второго к первому:

In [50]:
df.pivot_table(index = 'family_status', values = 'debt', aggfunc = ['count', 'sum', 'mean']) \
    .sort_values(by = ('mean', 'debt'), ascending = False) \
    .style.format({('mean', 'debt'): '{:.2%}'})

,count,sum,mean
,debt,debt,debt
family_status,,,
Не женат / не замужем,2810,274,9.75%
гражданский брак,4151,388,9.35%
женат / замужем,12339,931,7.55%
в разводе,1195,85,7.11%
вдовец / вдова,959,63,6.57%


**Вывод:** 
1. Овдовевшие заёмщики, чаще остальных категорий (6,6%), своевременно погашают кредиты.
2. Заёмщики, состоящие в `Гражданском браке`, являются самыми не надёжными по возврату заёмных средств (9,3%).

Обращает на себя внимание тот факт, что в предыдущем исследовании, самыми надёжными являлись заёмщики не имеющие детей. С учётом дополнительно полученных сведений, формируется вывод о том, что надёжный заёмщик не имеет детей и состоит или состоял в браке.

### 3.3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Сгруппируем данные по категории уровня дохода `total_income_category`, а также посчитаем общее количество заёмщиков, количество просроченных кредитов и отношение второго к первому:

In [51]:
df.pivot_table(index = 'total_income_category', values = 'debt', aggfunc = ['count', 'sum', 'mean']) \
    .sort_values(by = ('mean', 'debt'), ascending = False) \
    .style.format({('mean', 'debt') : '{:.2%}'})

,count,sum,mean
,debt,debt,debt
total_income_category,,,
E,22,2,9.09%
C,16015,1360,8.49%
A,25,2,8.00%
B,5042,356,7.06%
D,350,21,6.00%


**Вывод:** 
- Самая `высокая доля просрочек` у заёмщиков категории "E", с доходом до 30000 рублей
- Самая `низкая доля просрочек` у заёмщиков категории "D", с доходом от 30001 до 50000 рублей

Предположение о наличии зависимости возвратов кредита от уровня дохода заёмщиков, не подтверждается. Заёмщики с уровнем дохода от 50001 до 200000 склонны к неоплате задолженности так же, как и заёмщики с уровнем дохода до 30000 рублей

<div class="alert alert-block alert-success"> <b>Дополнительные выводы:</b>
<ol>
    <li>Заёмщики относящиеся категории "B" и "C" и имеющие доход от 50 т.р. до 1 млн., являются самыми активными участниками рынка кредитования и составляют основную долю заёщиков банка</li>
    <li>Заёмщикам относящимся к категории "C" выдано в 3 раза больше кредитов чем относящимся к категории "B", при этом, количество просроченных кредитов выше только на 20,3%. Это может указывать на ранее просчитанные риски Банка и принятые решения о расширении Банка своего влияние на конкретном сегменте рынке.</li>
        <li>Кредитные продукты Банка массово ориентированы на заёмщиков с доходами от 50 т. до 1 млн. и отсутсвтует достаточное количество банковских продуктов ориентированных на более состоятельных заёмщиков <i>(категория "A" доля выданных кредитов <b>менее 0,1%</b>)</i>, тем самым упускают экономическую выгоду от привлечения новых состоятельных клиентов.</li>
</ol>
</div>

### 3.4. Каким образом цель влияет на возврат кредита в срок?

Сгруппируем данные по цели `purpose_category`, а также посчитаем общее количество заёмщиков, количество просроченных кредитов и отношение второго к первому:

In [52]:
df.pivot_table(index = 'purpose_category', values = 'debt', aggfunc = ['count', 'sum', 'mean']) \
    .sort_values(by = ('mean', 'debt'), ascending = False) \
    .style.format({('mean', 'debt') : '{:.2%}'})

,count,sum,mean
,debt,debt,debt
purpose_category,,,
операции с автомобилями,4306,403,9.36%
получение образования,4013,370,9.22%
проведение свадьбы,2324,186,8.00%
операции с недвижимостью,10811,782,7.23%


**Вывод:** 
- Наиболее подвержено риску не возврата средств, кредитование на цели приобретения автомобиля (9,3%);
- Меньше всего рисков по не возврата средств, приходится на кредитование сделок с недвижимостью (7,3%);

По расчётам можно сделать вывод, что самыми проблемными выглядят кредиты на `операции с автомобилями` и `получение образования`.

### 3.5. Общий вывод

**В рамках реализации исследования были проведены следующие работы:**

- Произведён анализ полученных данных на предмет их целостности и возможности применения в исследовании.

- Произведена работа по нормализации информации (устранение выявленных аномалий, отклонений и дубликаты в данных, которые оказывали прямое влияние на качество и точность проводимого исследования).

- Произведено обогащение данными, необходимыми для повышения качества исследования.

- Проведён анализ влияющих факторов <i>(количество детей; семейное положение; уровень дохода и цели кредита)</i> на возврат кредита в срок.

- Произведено формирование качественных и количественных выводов исследования.

**Предположительные причины появления пропусков в данных:**

- Столбцы `days_employed` и `total_income` - имеют пустые значения, более 10% выборки данных, с учётом факта отсутствия данных в одних и тех же строках, данные пропуски могли появиться в следствии добавления в базу данных двух новых столбцов `days_employed` и `total_income`, либо при миграции данных между базами данных, либо в необязательности указания данных при первичном вводе. 
- `days_employed` - отрицательные значения в количественной переменной, является следствием неправильной настройки миграции данных.
- `children` - имеет отрицательное значение и значение 20 детей. Является ошибкой при вводе данных в систему.
- `education` - разный регистр заполнения категорийной переменной. Является ошибкой при вводе данных в систему.\
- Случайные повреждения данных молодыми сотрудниками компании, не имеющих достаточного опыта работы с БД и запросами к базе и тем не менее допущенными к "боевой" базе без должного опыта.

Для корректности ввода первичных данных в систему, рекомендуется ввести проверку вводимых данных

**Целесообразность использование медианного значения:** медианное значение выбрано для использования, т.к. эффективнее отрабатывает аномальные отклонения верхних и нижних значений выборки данных, по сравнению со средним значением.

**Вывод:**

Наименее рискованные заёмщики для банка, являются клиенты, которые которые соответствуют следующим критериям:
- Клиент обращается в банк за кредитом на приобретения недвижимости;
- У клиента уровень дохода относится к категории "D", от 30 т.р. до 50 т.р.;
- Состоит в законном браке или является вдовой/вдовцом и не имеет детей. 